# idea]
## - 배열을 이용하면 삽입, 삭제시 O(n)만큼 소요
## - linked list 이용
    - 배열의 크기가 큰 경우 linkeed list를 이용하는 것이 효율적
    - U, D 명령에서 O(X)
        - cmd에 등장하는 모든 X들의 합이 1,000,000 이하인 경우만 입력으로 주어진다고 하였으므로 효율성 테스트 통과 가능
    - C 명령에서 O(1)
    - Z 명령에서 O(1)
    - 단, 맨 앞, 맨 뒤에 삽입, 삭제해야하는 경우 주의


In [ ]:
from collections import deque

def solution(n, k, cmd):
    answer = ''

    table = dict() # element id, previous id, next id
    for i in range(n):
        if i == 0:
            table[i] = ("first", i + 1)
        elif i == n - 1:
            table[i] = (i - 1, "last")
        else:
            table[i] = (i - 1, i + 1)
    deleted = deque()

    current_id = k
    prev_id, next_id = table[k]

    for idx, c in enumerate(cmd):
        command = list(c.split())
        # 현재 선택된 행에서 X칸 위에 있는 행을 선택
        if command[0] == "U":
            count = int(command[1])
            while count != 0:
                prev_id, next_id = table[current_id]
                current_id = prev_id
                count -= 1

        # 현재 선택된 행에서 X칸 아래에 있는 행을 선택
        elif command[0] == "D":
            count = int(command[1])
            while count != 0:
                prev_id, next_id = table[current_id]
                current_id = next_id
                count -= 1
        # 현재 선택된 행을 삭제한 후, 바로 아래 행을 선택
        # 단, 삭제된 행이 가장 마지막 행인 경우 바로 윗 행을 선택
        elif command[0] == "C":
            prev_id, next_id = table[current_id]
            del table[current_id]
            deleted.append((current_id, (prev_id, next_id)))

            # 삭제하는 행이 첫번째 행인 경우
            if prev_id == "first":
                next_before, next_after = table[next_id]
                table[next_id] = "first", next_after
                current_id = next_id
            # 삭제하는 행이 마지막 행인 경우
            elif next_id == "last":
                prev_before, prev_after = table[prev_id]
                table[prev_id] = prev_before, "last"
                current_id = prev_id
            else:
                prev_before, prev_after = table[prev_id]
                table[prev_id] = prev_before, next_id
                next_before, next_after = table[next_id]
                table[next_id] = prev_id, next_after
                current_id = next_id
        # 가장 최근에 삭제된 행을 원래대로 복구
        # 단, 현재 선택된 행은 바뀌지 않습니다.
        elif command[0] == "Z":
            element_id, (prev_id, next_id) = deleted.pop()
            table[element_id] = (prev_id, next_id)

            if prev_id == "first": # previously located in the front
                next_before, next_after = table[next_id]
                table[next_id] = element_id, next_after
            elif next_id == "last": # previously located in the last
                prev_before, prev_after = table[prev_id]
                table[prev_id] = prev_before, element_id
            else:
                prev_before, prev_after = table[prev_id]
                table[prev_id] = prev_before, element_id
                if prev_after == "last": # inserted end of the last element
                    table[element_id] = (prev_id, "last")

                next_before, next_after = table[next_id]
                table[next_id] = element_id, next_after
                if next_before == "first": # inserted infront of the first element
                    table[element_id] = ("first", next_id)

    deleted = dict(deleted)
    for i in range(n):
        if i in deleted.keys():
            answer += "X"
        else:
            answer += "O"

    return answer